DevOps Toolsumgebung
====================


Planen
----------

![](https://camo.githubusercontent.com/0b2807bac30d4fbd405988d283da12f35ae4973b/68747470733a2f2f6b616e626f6172642e6f72672f6173736574732f696d672f626f6172642e706e67)

Kanboard ist eine Open-Source Kanban-Projektmanagement-Software.

Es erlaubt Aufgaben in Projekten und Sprints (Swimlanes) zu managen. Die Columns können frei angepasst werden.

Starten:

In [ ]:
! kubectl create -f https://raw.githubusercontent.com/mc-b/duk/master/devops/kanboard.yaml

Web Oberfläche mittels [Cluster-IP:32200](http://localhost:32200) anwählen. Username/Password ist `admin`.

***
Source Code Management (optional)
----------------------

![](https://gogs.io/img/screenshots/4.png)

Gogs ist ein einfacher Git Server mit Weboberfläche. Erinnert stark an github.

Starten:

In [ ]:
! kubectl create -f https://raw.githubusercontent.com/mc-b/duk/master/devops/gogs.yaml

Web Oberfläche mittels [Cluster-IP:32300](http://localhost:32300) anwählen.	

Werte bei **Installationsschritte für den ersten Start** wie folgt anpassen:
* Datenbanktyp: SQLite3
* Pfad: /data/gogs.db
* Anwendungs-URL: IP-Adresse und Port Cluster, z.B. http://localhost:32300/

Einstellungen, wenn es sich um einen nicht frei zugänglichen Server handelt:
* Server und sonstige Einstellungen 
    * Registrierung deaktivieren: true
    * Captcha aktivieren: false 
* Administrator Einstellungen: Admin User inkl. Password einrichten.

Ansonsten erhält der erste User wo sich registriert, Adminstrator Rechte.

*** 

Mittels `+` - `Neue Migration` bestehendes Repository von GitHub kopieren
* Adresse kopieren: [https://github.com/mc-b/bpmn-tutorial.git](https://github.com/mc-b/bpmn-tutorial.git)
* Repository-Name: bpmn-tutorial

Vorgang wiederholen für:
* [https://github.com/mc-b/microservice-kubernetes](https://github.com/mc-b/microservice-kubernetes)
* [https://github.com/mc-b/microservice-kafka.git](https://github.com/mc-b/microservice-kafka.git)
* [https://github.com/mc-b/SCS-ESI.git](https://github.com/mc-b/SCS-ESI.git)


***
Build System Maven (optional)
--------------------------------------------

Maven ist ein Tool, mit dem Sie jetzt jedes Java-basierte Projekt erstellen und verwalten können. 

Um dies zu Demonstrieren, erstellen wir ein neues Projekt mittels des `archetypeArtifactId=maven-archetype-quickstart`, compilieren und starten es:

In [ ]:
! docker run -it --rm -v /src:/src -w /src/ maven:3-alpine mvn archetype:generate -DgroupId=com.mycompany.app -DartifactId=my-app -DarchetypeArtifactId=maven-archetype-quickstart -DarchetypeVersion=1.4 -DinteractiveMode=false

In [ ]:
! docker run -it --rm -v /src:/src -w /src/my-app maven:3-alpine mvn clean package            

In [ ]:
! docker run -it --rm -v /src:/src -w /src/my-app maven:3-alpine java -cp target/my-app-1.0-SNAPSHOT.jar com.mycompany.app.App

Den erstellten Sourcecode und die Artefakte kopieren wir ins aktuelle Verzeichnis 

In [ ]:
! kubectl  exec deployment/dind tar cf - /src/my-app | tar xvf -
! find src/my-app

Um das so erstellte Projekt besser zu verstehen, schauen wir das Buildfile `pom.xml` und das Main-Programm `App.java` an:

In [ ]:
! cat src/my-app/pom.xml
! echo "-------------------"
! cat src/my-app/src/main/java/com/mycompany/app/App.java

Der Einfachheit überspringen wir die weiteren Schritte wie:
* Abstellen des Sourcecodes im Source Code Management (git)
* Abstellen des Artefaktes im [Zentralen Maven Repository](https://mvnrepository.com/)

und gehen direkt zum Erstellen eines Container Images

Dazu Erstellen wir zuerst ein Dockerfile. 

Das Dockerfile
* ist abgeleitet vom Base Image `openjdk:8-jre`
* wird ergänzt um das Compilierte Programm, aufbereitet in der JAR-Datei `target/my-app-1.0-SNAPSHOT.jar` 
* um dieses dann im Container starten zu können

In [ ]:
%%bash
cat <<%EOF% >src/my-app/Dockerfile
FROM openjdk:8-jre
COPY target/my-app-1.0-SNAPSHOT.jar /opt
CMD [ "java", "-cp", "/opt/my-app-1.0-SNAPSHOT.jar", "com.mycompany.app.App" ]
%EOF%
cd src/my-app
docker build -t my-app  .

Zum Schluss schauen wir uns das erstellte Container Image an und starten dieses:

In [ ]:
! docker image ls my-app
! docker history my-app
! docker run my-app

***
Artefakte und Container Repository/Registry
-------------------------------------------

Neben Artefakten wie Programme, Libraries etc. müssen heute auch Container Images gespeichert werden.

Die nachfolgenden Befehle starten eine Docker Registry und stellen diese unter [http://localhost:32580](http://localhost:32580 ) zur Verfügung.


In [ ]:
! kubectl apply -f https://raw.githubusercontent.com/mc-b/duk/master/registry/registry2.yaml
! kubectl apply -f https://raw.githubusercontent.com/mc-b/duk/master/registry/registry2-frontend.yaml

Sobald diese gestartet ist können wir unser Container Image dort abstellen

    docker tag my-app docker-registry:5000/my-app
    docker push docker-registry:5000/my-app
    

*** 
Continuous Integration
----------------------

![](https://jenkins.io/images/blueocean/blueocean-successful-pipeline.png)

- - -

Nachdem wir gesehen haben, wie viele Schritte manuell durchgeführt werden müssen, ist es an der Zeit diese zu automatisieren.

Dazu verwenden wir 

*Continuous Integration (CI) (auch fortlaufende oder permanente Integration) ein Begriff aus der Software-Entwicklung, der den Prozess des fortlaufenden Zusammenfügens von Komponenten zu einer Anwendung beschreibt.*

Dazu müssen wir zuerst für unsere `my-app` Applikation ein CI-Script schreiben und dieses im Source Code Management (git) abstellen.

In [ ]:
%%bash
cat <<%EOF% >src/my-app/Jenkinsfile
pipeline {
    agent {
        docker {
            image 'maven:3-alpine'
            args '-v /root/.m2:/root/.m2'
        }
    }
    stages {
        stage('Build') {
            steps {
                sh 'mvn -B -DskipTests clean package'
                archiveArtifacts 'target/*.jar'
                stash includes: 'target/*.jar', name: 'jar'
            }
        ...
        stage('Build Images') {
            agent none
            steps {
                unstash 'jar'
                sh 'docker build -t my-app .'
            }
        }
    }
}
%EOF%

Anschliessend starten wir das CI System, hier Jenkins.

! kubectl create -f https://raw.githubusercontent.com/mc-b/duk/master/devops/jenkins.yaml

Die Oberfläche wird fix auf Port 32100 zugewiesen und via [Cluster-IP:32100](http://localhost:32100) erreichbar.

Initialer Username und Password ist `admin`

Oberfläche Blue Ocean anwählen und neue Pipeline mittels `git` und
Repository-URL, z.B. von `gogs` [http://localhost:32300/root/microservice-kubernetes.git](http://localhost:32300/root/microservice-kubernetes.git) anlegen. 